In [1]:
# Import Packages:
from __future__ import print_function

import os, sys
from six.moves import cPickle as pickle
from six.moves import range
from collections import defaultdict


# Model Packages import
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix

# Get the current working directory
cwd = os.getcwd()
mydir = os.path.abspath(os.path.join(cwd, ".."))
sys.path.append(mydir)
from DataGenerator import genTrainValidFolds


# Packages for plot:
import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()
    
    
def reshape_data(dataset, labels=None, imageSize=32, numChannels=1, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape((-1,imageSize,imageSize,numChannels)) # To reshape the
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        if labels:
            numLabels = len(np.unique(labels))
            labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape((-1,imageSize,imageSize,numChannels)) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        if labels:
            numLabels = len(np.unique(labels))
            labels = (np.arange(numLabels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [3]:
def accuracy(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])


def confusionMatrix(predictions, labels):
    # Both the predictions and the labels should be in One-Hot vector format.
    return (pd.crosstab(np.argmax(labels, 1), np.argmax(predictions, 1), rownames=['True'], colnames=['Predicted'], margins=True))

# Build the Train, CrossValid and Test Graph:

The below modules build a simple convolutional neural network graph.

In [4]:
class BuildConvNet():

    def __init__(self, params):
        self.imageSize = params['imageSize'],

        self.numFolds = params['imageSize'],
        
        # Network Parameters
        self.numLabels = params['imageSize'],
        self.numChannels = params['imageSize'],               # numChannels -> For grayscale =1, for RGB =3
                
        # Convolution filter params
        self.conv11_Kernel = params['imageSize'],             # Size of kernel for the first convolution layer
        self.conv11_Stride = params['imageSize'],             # Strides for the first convolution layer filter 
        self.conv11_Depth = params['imageSize'],              # Number of kernels for the first convolution layer
        self.conv12_Kernel = params['imageSize'],             # Size of kernel for the first convolution layer
        self.conv12_Stride = params['imageSize'],             # Strides for the first convolution layer filter 
        self.conv12_Depth = params['imageSize'],              # Number of kernels for the first convolution layer
        self.pool1_Kernel = params['imageSize'],              # Size of kernel for the first Pooling layer
        self.pool1_Stride = params['imageSize'],              # Strides for the first Pooling layer filter
        
        self.conv21_Kernel = params['imageSize'],             # Size of kernel for the first convolution layer
        self.conv21_Stride = params['imageSize'],             # Strides for the first convolution layer filter 
        self.conv21_Depth = params['imageSize'],              # Number of kernels for the first convolution layer
        self.conv22_Kernel = params['imageSize'],             # Size of kernel for the first convolution layer
        self.conv22_Stride = params['imageSize'],             # Strides for the first convolution layer filter 
        self.conv22_Depth = params['imageSize'],              # Number of kernels for the first convolution layer
        self.pool2_Kernel = params['imageSize'],              # Size of kernel for the first Pooling layer
        self.pool2_Stride = params['imageSize'],              # Strides for the second Pooling layer filte
        
        # Fully connected layer params
        self.numHidden1 = params['imageSize'],                # numHiddenUnits for the fully connected layerr
        self.numHidden2 = params['imageSize'],                # numHiddenUnits for the fully connected layer
        
        # Batch and Learning Variables     
        self.epochs = params['imageSize'],
        self.batchSize = params['imageSize'],
        self.keepProb = params['imageSize'],
        self.optimizerParam = params['imageSize'] 


        
        # WEIGHTS AND BIASES
        self.weights = {
            'cv11_wght': tf.Variable(tf.random_normal([self.conv11_Kernel, self.conv11_Kernel, self.numChannels, self.conv11_Depth], seed=8753)),
            'cv12_wght': tf.Variable(tf.random_normal([self.conv12_Kernel, self.conv12_Kernel, self.conv11_Depth, self.conv12_Depth], seed=231)),
            'cv21_wght': tf.Variable(tf.random_normal([self.conv21_Kernel, self.conv21_Kernel, self.conv12_Depth, self.conv21_Depth], seed=4221)),
            'cv22_wght': tf.Variable(tf.random_normal([self.conv22_Kernel, self.conv22_Kernel, self.conv21_Depth, self.conv22_1Depth], seed=680)),
            'fc2_wght': tf.Variable(tf.random_normal([self.numHidden1, self.numHidden2], seed=598)),
            'out_wght': tf.Variable(tf.random_normal([self.numHidden2, self.numLabels], seed=332))
        }

        self.biases = {
            'cv11_bias': tf.Variable(tf.random_normal([self.conv11_Depth], seed=8767)),
            'cv12_bias': tf.Variable(tf.random_normal([self.conv12_Depth], seed=9887)),
            'cv21_bias': tf.Variable(tf.random_normal([self.conv21_Depth], seed=8767)),
            'cv22_bias': tf.Variable(tf.random_normal([self.conv22_Depth], seed=9887)),
            'fc1_bias': tf.Variable(tf.random_normal([self.numHidden1], seed=4323)),
            'fc2_bias': tf.Variable(tf.random_normal([self.numHidden2], seed=4348)),
            'out_bias': tf.Variable(tf.random_normal([self.numLabels], seed=9878))
        }


    def convLayer(self, x, w, b, s=1, nlModel='RELU'):
        x = tf.nn.conv2d(x, w, [1,s,s,1], padding='SAME') # Same padding
        x = tf.nn.bias_add(x, b)
        if nlModel == 'RELU':
            return tf.nn.relu(x)
        elif nlModel == 'LOGIT':
            return tf.sigmoid(x)


    def poolLayer(self, x, k=2, s=2, poolType='MAX'):
        if poolType=='MAX':
            return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,s,s,1], padding='SAME')
        elif poolType == 'AVG':
            return tf.nn.avg_pool(data, ksize=[1,k,k,1], strides=[1,s,s,1], padding='SAME')

        
    def fcLayer(self, x, w, b, keepProb=None, nModel='RELU'):
        x = tf.matmul(x, w) + b
        if nModel=='RELU':
            x = tf.nn.relu(x)
        if nModel=='LOGIT':
            x = tf.sigmoid(x)
        if keepProb:
            return tf.nn.dropout(x, keepProb, seed=6162)
        else:
            return x
        
        
    def outputLayer(self, x, w, b):
        x = tf.matmul(x, w) + b
        return x, tf.nn.softmax(x)
    
            
    # Convolutional Layer (Includes the convLayer and the max pool layer)
    def convFeatureExtractor(self, x):
        # Convolution and Pooling Layer 1
        conv1 = self.convLayer(x, self.weights['cv11_wght'], self.biases['cv11_bias'], s=self.conv11_Stride)
        conv1 = self.convLayer(conv1, self.weights['cv12_wght'], self.biases['cv12_bias'], s=self.conv12_Stride)
        conv1 = self.poolLayer(conv1, k=self.pool1_Kernel, s=self.pool1_Stride)

        # Convolution and Pooling Layer 2
        conv2 = self.convLayer(conv2, self.weights['cv21_wgth'], self.biases['cv21_bias'], self.conv21_Stride)
        conv2 = self.convLayer(conv2, self.weights['cv22_wgth'], self.biases['cv22_bias'], self.conv22_Stride)
        conv2 = self.poolLayer(conv2, k=self.pool2_Kernel, s=self.pool2_Stride)
        
        # Get the features in flattened fashion
        poolShape = conv2.get_shape().as_list()[1]
        numConvFeatures = poolShape*poolShape*self.conv2Depth
        convFeatures = tf.reshape(conv2, [-1, numConvFeatures])
        
        return convFeatures, numConvFeatures
        
        
    # Create the training Graph Lineage
    def trainGraph(self):
        trainData = tf.placeholder(tf.float32, [None, self.imageSize, self.imageSize, self.numChannels])
        trainLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # Convolutional Layer
        convFeatures, numConvFeatures = self.convFeatureExtractor(trainData)
        
        # Fully connected layer : Reshape conv2 output to fit fully connected layer input
        self.weights['fc1_wght'] =  tf.Variable(tf.random_normal([numConvFeatures, self.numHidden1], seed=6663))
        fc1State = self.fcLayer(convFeatures, self.weights['fc1_wght'], self.biases['fc1_bias'], keepProb=self.keepProb['fc1'], nModel='RELU')  # = 0.5
        
        # Fully connected layer : Layer 2
        fc2State = self.fcLayer(fc1State, self.weights['fc2_wght'], self.biases['fc2_bias'], keepProb=self.keepProb['fc2'], nModel='RELU')  # 0.75

        # Output, or the Softmax layer
        pred, outputState = self.outputLayer(fc2State, self.weights['out_wght'], self.biases['out_bias'])
        
        # Loss function and Optimization
        lossCE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=trainLabels))
        
        if self.optimizerParam['optimizer'] == 'ADAM':
            optimizer = tf.train.AdamOptimizer(learning_rate=self.optimizerParam['learning_rate']).minimize(lossCE)
        elif self.optimizerParam['optimizer'] == 'RMSPROP':
            optimizer = tf.train.RMSPropOptimizer(learning_rate=self.optimizerParam['learning_rate'], 
                                                  momentum=self.optimizerParam['momentum']).minimize(lossCE)  # 0.0006  # 0.8
        else:
            print ("Your provided optimizers do not match with any of the initialized optimizers: .........")
            return None

        
        return dict(
            trainData=trainData, 
            trainLabels=trainLabels, 
            optimizer=optimizer, 
            lossCE=lossCE,
            trainPred = outputState,
            wghtNew = self.weights
        )
    
    
    # Create the Validation Graph Lineage
    def validGraph(self):

        validData = tf.placeholder(tf.float32, [None, self.imageSize, self.imageSize, self.numChannels])
        validLabels = tf.placeholder(tf.float32, [None, self.numLabels])

        # Convolution and Pooling Layer 1
        vconvFeatures, numConvFeatures = self.convFeatureExtractor(validData)
        # Convolution and Pooling Layer 1
#         vconv1 = self.convLayer(validData, self.weights['cv1_wght'], self.biases['cv1_bias'], s=self.conv1Stride)
#         vconv1 = self.poolLayer(vconv1, k=self.pool1Kernel)

#         # Convolution and Pooling Layer 2
#         vconv2 = self.convLayer(vconv1, self.weights['cv2_wgth'], self.biases['cv2_bias'], self.conv2Stride)
#         vconv2 = self.poolLayer(vconv2, k=self.pool2Kernel)
        
#         # Get the features in flattened fashion
#         vpoolShape = vconv2.get_shape().as_list()[1]
#         vnumConvFeatures = vpoolShape*vpoolShape*self.conv2Depth
#         vconvFeatures = tf.reshape(vconv2, [-1, vnumConvFeatures])
        
        
        # Fully connected layer/Hidden Layer 2 : Reshape conv2 output to fit fully connected layer input
        vFc1State = self.fcLayer(vconvFeatures, self.weights['fc1_wght'], self.biases['fc1_bias'], nModel='RELU')
        
        # Fully connected layer/Hidden layer 2
        vFc2State = self.fcLayer(vFc1State, self.weights['fc2_wght'], self.biases['fc2_bias'], nModel='RELU')


        # Output Layer or the softmax layer
        vPred, vOutState = self.outputLayer(vFc2State, self.weights['out_wght'], self.biases['out_bias'])


        return dict(
            validData = validData,
            validLabels = validLabels,
            validPred = vOutState
        )

# Execute the Train, CrossValid and Test graph and evaluate Models:

The below modules executes the graph given the input parameters and provides the training accuracy, validation accuracy and confusion matrix.

In [5]:
class SessionExec():
    
    def __init__(self, featureDIR, params):
        self.featureDIR = featureDIR
        
        self.numFolds = params['numFolds']
        self.epochs = params['epochs']
        self.batchSize = params['batchSize']
        self.imageSize = params['imageSize']
        self.numChannels = params['numChannels']
        
        self.prevWghtDict = {}
        self.wghtChngDict = defaultdict(list)
        self.meanValidAcc = 0
        self.loss_PerEpoch_PerFold = defaultdict(list)
        self.trainacc_PerEpoch_PerFold = defaultdict(list)
        self.validacc_PerEpoch_PerFold = defaultdict(list)
        
        
        
    def captureWghtChange(self, wghtDict):   
        for layer, wght_arr in wghtDict.items():
#             print (layer)
#             print (np.sum(np.abs(wght_arr-self.prevWghtDict[layer])))
#             print ('********************')
            self.wghtChngDict[layer].append(np.sum(np.abs(wght_arr-self.prevWghtDict[layer])))
    
    
    
    def trainModel(self, trainDataIN, trainLabelsIN, sess):    
        numBatches = int(trainDataIN.shape[0]/self.batchSize)
        for numBatch in np.arange(numBatches):
#                     print ('Batch no: ', numBatch)
#                     print (numBatch*self.batchSize,  (numBatch+1)*self.batchSize)
            batchData = trainDataIN[numBatch*self.batchSize : (numBatch+1)*self.batchSize]
            batchLabels = trainLabelsIN[numBatch*self.batchSize : (numBatch+1)*self.batchSize]

            feed_dict = {self.trainGraphDict['trainData']: batchData,
                         self.trainGraphDict['trainLabels']: batchLabels
#                              self.trainGraphDict['keep_prob']: 0.7
                    }

            _, loss, tpred, wght = sess.run([self.trainGraphDict['optimizer'],
                                        self.trainGraphDict['lossCE'],
                                        self.trainGraphDict['trainPred'],
                                        self.trainGraphDict['wghtNew']], feed_dict=feed_dict)
            
            if any(self.prevWghtDict):
                self.captureWghtChange(wght)
                self.prevWghtDict = wght
            else:
                self.prevWghtDict = wght
                
                
            if ((numBatch+1)%10 == 0) or ((numBatch+1) == numBatches): 
                tacc = accuracy(tpred, batchLabels)
                print ("Fold: " + str(self.foldNUM+1) + 
                       ", Epoch: " + str(self.epoch+1)+ 
                       ", Mini Batch: " + str(numBatch+1) + 
                       ", Loss= " + "{:.6f}".format(loss) + 
                       ", Training Accuracy= " + "{:.5f}".format(tacc))
    
        return loss, tacc
    
    
    def validModel(self, validDataIN, validLabelsIN, sess):
        feed_dict = {self.validGraphDict['validData']: validDataIN,
                     self.validGraphDict['validLabels']: validLabelsIN
#                              self.trainGraphDict['keep_prob']: 0.7
                }

        vpred = sess.run(self.validGraphDict['validPred'], feed_dict=feed_dict)
        
        vacc = accuracy(vpred, validLabelsIN)
        print ("Fold: " + str(self.foldNUM+1) + 
                ", Epoch: " + str(self.epoch+1)+ 
                ", Validation Accuracy= " + "{:.5f}".format(vacc))
        
        return vpred, vacc 
        
        
    def execute(self):
        meanValidAcc = 0
        
        for foldNUM, (trainDataIN, trainLabelsIN, validDataIN, validLabelsIN, labelDict) in enumerate(genTrainValidFolds(self.featureDIR, oneHot=True)):   
            print ('')
            print ('##########################################################################################')
            trainDataIN, _ = reshape_data(trainDataIN, imageSize=self.imageSize, numChannels=self.numChannels)                                       
            validDataIN, _ = reshape_data(validDataIN, imageSize=self.imageSize, numChannels=self.numChannels)
            
            print ('')
            print ('Validation Data and Labels shape: ', validDataIN.shape, validLabelsIN.shape)
            print ('Training Data and Labels shape: ', trainDataIN.shape, trainLabelsIN.shape)
            print ('The Label Dictionary is given as: ', labelDict)
            print ('')
            
            self.foldNUM = foldNUM

            reset_graph()

            # Create a object encapsulating the graph lineage
            objCNN = BuildConvNet(params)
            self.trainGraphDict = objCNN.trainGraph()
            self.validGraphDict = objCNN.validGraph()
            
            with tf.Session() as sess:
                sess.run(tf.initialize_all_variables())

                for epoch in range(self.epochs): 
                    self.epoch = epoch
                    
                    # Train The model for Epochs
                    loss, tacc = self.trainModel(trainDataIN, trainLabelsIN, sess)
                    print ('')
                    
                    # Validate the model for Epochs
                    vpred, vacc = self.validModel(validDataIN, validLabelsIN, sess)
                    print ('')
                    
                    self.loss_PerEpoch_PerFold[foldNUM+1].append(loss)
                    self.trainacc_PerEpoch_PerFold[foldNUM+1].append(tacc)
                    self.validacc_PerEpoch_PerFold[foldNUM+1].append(vacc)
                    
                    if (epoch+1)%10 ==0 or ((epoch+1) == self.epochs):
                        validCM = confusionMatrix(validLabelsIN,vpred)
                        print ('Confusion Matrix CrossValid Set')
                        print (validCM)
                        print ('')
            
            self.meanValidAcc += vpred
            
            if foldNUM == self.numFolds-1:
                break
                
        return (self.wghtChngDict, 
                self.meanValidAcc/self.numFolds, 
                self.loss_PerEpoch_PerFold, 
                self.trainacc_PerEpoch_PerFold, 
                self.validacc_PerEpoch_PerFold)
                
  

# MODEL 1: Standarized Feature Set (ADAM)

In [ ]:
STDbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/STD/batchData/"
EDGbatch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/EDG/batchData/"
HOGp1batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp1/batchData/"   # dim =162
HOGp2batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp2/batchData/"   # dim =576
HOGp3batch_dir = "/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/Model-Sardhendu/HOGp3/batchData/"

featureDIR = STDbatch_dir

params = dict(imageSize = 32,
              seed = 2316,                 # Set the seeds for the random weights generator
              numFolds = 1,
              
              # Network Parameters
              numLabels = 2,
              numChannels = 1,             # numChannels -> For grayscale =1, for RGB =3
              
              # Convolution filter params
              conv11_Kernel = 5,             # Size of kernel for the first convolution layer
              conv11_Stride = 2,             # Strides for the first convolution layer filter 
              conv11_Depth = 16,             # Number of kernels for the first convolution layer
              conv12_Kernel = 3,             # Size of kernel for the first convolution layer
              conv12_Stride = 2,             # Strides for the first convolution layer filter 
              conv12_Depth = 32,             # Number of kernels for the first convolution layer
              pool1_Kernel = 2,             # Size of kernel for the first Pooling layer
              pool1_Stride = 2,             # Strides for the first Pooling layer filter
              
              conv21_Kernel = 5,             # Size of kernel for the first convolution layer
              conv21_Stride = 2,             # Strides for the first convolution layer filter 
              conv21_Depth = 16,             # Number of kernels for the first convolution layer
              conv22_Kernel = 3,             # Size of kernel for the first convolution layer
              conv22_Stride = 2,             # Strides for the first convolution layer filter 
              conv22_Depth = 32,             # Number of kernels for the first convolution layer
              pool2_Kernel = 2,             # Size of kernel for the first Pooling layer
              pool2_Stride = 2,             # Strides for the second Pooling layer filter

              
              numHidden1 = 1024,           # numHiddenUnits for the fully connected layerr
              numHidden2 = 1024,            # numHiddenUnits for the fully connected layer
              
              # Learning Variables     
              epochs = 20,
              batchSize = 256,
        
              keepProb = dict(fc1=0.5, fc2=0.7),
#               optimizerParam = dict(optimizer='RMSPROP', learning_rate=0.0008, momentum=0.8)
              optimizerParam = dict(optimizer='ADAM', learning_rate=0.004)  # at 0.003 descents slowely, 
        )

# print (params)

(out, meanValidAcc, lossPerEpochPerFold, taccPerEpochPerFold, vaccPerEpochPerFold) = SessionExec(featureDIR, params).execute() 

print ('The mean Valid Accuracy is : ', meanValidAcc)
# print ('The mean Valid Accuracy is : ', )
# print ('The mean Valid Accuracy is : ', )
# print ('The mean Valid Accuracy is : ', )


##########################################################################################

Validation Data and Labels shape:  (1000, 32, 32, 1) (1000, 2)
Training Data and Labels shape:  (9000, 32, 32, 1) (9000, 2)
The Label Dictionary is given as:  {0: 'trainDataAirplane.pickle', 1: 'trainDataCat.pickle'}

Fold: 1, Epoch: 1, Mini Batch: 10, Loss= 482034.500000, Training Accuracy= 64.45312
Fold: 1, Epoch: 1, Mini Batch: 20, Loss= 166031.250000, Training Accuracy= 73.43750
Fold: 1, Epoch: 1, Mini Batch: 30, Loss= 120033.960938, Training Accuracy= 67.96875
Fold: 1, Epoch: 1, Mini Batch: 35, Loss= 106169.234375, Training Accuracy= 73.04688

Fold: 1, Epoch: 1, Validation Accuracy= 79.10000

Fold: 1, Epoch: 2, Mini Batch: 10, Loss= 69455.203125, Training Accuracy= 71.87500
Fold: 1, Epoch: 2, Mini Batch: 20, Loss= 57183.703125, Training Accuracy= 73.43750
Fold: 1, Epoch: 2, Mini Batch: 30, Loss= 57612.492188, Training Accuracy= 75.39062
Fold: 1, Epoch: 2, Mini Batch: 35, Loss= 43618.523438